# Location

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
sede_dimension = pd.read_sql_table("sede", OLTP_connection)
city_dimension = pd.read_sql_table("ciudad", OLTP_connection)

In [4]:
sede_dimension.head(1)

,sede_id,nombre,direccion,telefono,nombre_contacto,ciudad_id,cliente_id
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4


In [5]:
city_dimension.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [6]:
location_dimension = sede_dimension.merge(city_dimension, on="ciudad_id", how="left")

In [7]:
location_dimension = location_dimension.rename(
	columns={
		"sede_id": "location_id",
		"nombre_x": "location_name",
		"tipo_sede": "location_type",
		"nombre_y": "location_city",
		"departamento_id": "location_region",
	}
)

In [8]:
location_dimension = location_dimension[
	[
		"location_id",
		"location_name",
		# "location_type",
		"location_city",
		"location_region",
	]
]

In [9]:
location_dimension.head(2)

,location_id,location_name,location_city,location_region
0,10,FARALLONES /123,CALI,1
1,11,REMEDIOZ/ 123,CALI,1


## Load

In [10]:
location_dimension.to_sql(
	"LOCATION_DIMENSION", OLAP_connection, if_exists="replace", index=False
)

52